In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from IPython.display import clear_output
import math, random, torch, collections, time, networkx as nx, matplotlib.pyplot as plt
import copy
import pandas as pd
import numpy as np
from numbers import Number
import scipy as sc
import torch.nn as nn
import torch.nn.functional as F
import torch
import torch_geometric as tg
import torch_geometric.transforms as T


# Helper function for visualization.
%matplotlib inline
import sys,os
sys.path.append('../')
from torch_geometric.transforms import RandomLinkSplit, RandomNodeSplit
from io_utils.visualisation import *
from train_utils.training import *
#from layer2 import GaussianSample


#### Creating a synthetic graph framework to test (a) Edge prediction (b) Robustness and (c) Transferability
from torch_geometric.datasets import Planetoid
from torch_geometric.transforms import NormalizeFeatures

dataset_name = 'Cora'
dataset = Planetoid(root='../../data/Planetoid', name=dataset_name, transform=NormalizeFeatures())
###
print()
print(f'Dataset: {dataset}:')
print('======================')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}')

data = dataset[0]  # Get the first graph object.

print()
print(data)
print('===========================================================================================================')

# Gather some statistics about the graph.
print(f'Number of nodes: {data.num_nodes}')
print(f'Number of edges: {data.num_edges}')
print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')
print(f'Number of training nodes: {data.train_mask.sum()}')
print(f'Training node label rate: {int(data.train_mask.sum()) / data.num_nodes:.2f}')
print(f'Contains isolated nodes: {data.has_isolated_nodes()}')
print(f'Contains self-loops: {data.has_self_loops()}')
print(f'Is undirected: {data.is_undirected()}')



###### geometric properties of the dataset

nodes = {k:  torch.sum(data.edge_index[0,:] ==k).numpy() for k in np.arange(data.x.shape[0])}


In [ ]:
#### Compute the edge weights
import copy
data = dataset[0]  # Get the first graph object.
data.edge_weight = torch.ones(data.edge_index.shape[1])
transform = T.GDC(
        self_loop_weight=1,
        normalization_in='row',
        normalization_out='row',
        diffusion_kwargs=dict(method='ppr', alpha=0.15),
        sparsification_kwargs=dict(method='topk', k=30, dim=0),
        exact=True,
    )

data2 = transform(copy.deepcopy(data))
    
    
# edges2, edge_weights = transform(copy.deepcopy(data)).transition_matrix(edge_index=data.edge_index,
#                          edge_weight=torch.ones(data.edge_index.shape[1]),
#                          num_nodes=data.num_nodes, normalization= 'row')
plt.hist(list(data2.edge_attr.numpy()))

In [ ]:
(data2.edge_attr ==0).sum()

In [ ]:
data2

In [ ]:
edge_index, edge_weight = add_remaining_self_loops(
                data.edge_index, torch.ones((data.edge_index.size(1), )), 1., data.num_nodes)
row, col = edge_index[0], edge_index[1]
deg = scatter_add(edge_weight, col, dim=0, dim_size=data.num_nodes)

In [ ]:
new_rows = []
new_cols = []
weights = []
sigmas = []
n_eff_neighbours = []
MAX_N_NEIGHBOURS = 50
for u in range(data.num_nodes):
    index = ((newA.row == u) & (newA.data >1e-5))
    new_rows+= list(newA.row[index])
    new_cols+= list(newA.col[index])
    #### want to find the appropriate scaling factor
    dist_row = newA.data[index]
    func = lambda sigma: k(prob_high_dim(sigma, dist_row))
    binary_search_result = sigma_binary_search(func, MAX_N_NEIGHBOURS) #### Maybe we should have a varying number of neighbours here
    sigmas += [binary_search_result]
    weights += list(prob_high_dim(binary_search_result, dist_row))
    n_eff_neighbours += [k(prob_high_dim(binary_search_result, dist_row))]
print(np.mean(weights))

In [ ]:
plt.hist(weights)

In [ ]:
### To compare against the usual weights
# from torch_geometric.utils import add_remaining_self_loops
# from torch_scatter import scatter_add 

# add_self_loops=True
# fill_value = 1
# alpha = 0.5

# edge_index0 = data.edge_index
# edge_weight = torch.ones((edge_index0.size(1), ))


# num_nodes = data.num_nodes
# edge_index, edge_weight = add_remaining_self_loops(
#                 edge_index0, edge_weight, fill_value, num_nodes)

# row, col = edge_index[0], edge_index[1]
# deg = scatter_add(edge_weight, col, dim=0, dim_size=num_nodes)
# deg_inv_sqrt = deg.pow_(-alpha)
# deg_inv_sqrt.masked_fill_(deg_inv_sqrt == float('inf'), 0)
# L = deg_inv_sqrt[row] * edge_weight * deg_inv_sqrt[col]
# #L = deg_inv_sqrt[row] * edge_weight #* deg_inv_sqrt[col]
# print(L)
# plt.hist(L.numpy())

In [ ]:
# from torch_geometric.utils import to_scipy_sparse_matrix, from_scipy_sparse_matrix
# L = deg_inv_sqrt[row] * edge_weight
# P = to_scipy_sparse_matrix(edge_index, edge_attr=L, num_nodes=data.num_nodes)
# A = P + P.T - P.multiply(P.T)
# d = from_scipy_sparse_matrix(A)
# plt.hist(d[1].numpy())

In [ ]:
def prob_low_dim(Y, YY, a=1., b=1.):
    """
    Compute matrix of probabilities q_ij in low-dimensional space
    """
    inv_distances = torch.power(1 + a * torch.sum(torch.square(Y-YY))**b, -1)
    return inv_distances

In [ ]:
class GCNter(nn.Module): # in_dim, hid_dims, out_dim, normalize=True
    def __init__(self, input_dim, hidden_dim, output_dim, n_layers, end='linear',
    activation='relu', slope=.1, device='cpu', normalize=True):
        super().__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.end = end
        self.n_layers = n_layers
        self.device = device
        self.propagate = APPNP(K=1, alpha=0)
        self.normalize = normalize
        
        if isinstance(hidden_dim, Number):
            self.hidden_dim = [hidden_dim] * (self.n_layers - 1)
        elif isinstance(hidden_dim, list):
            self.hidden_dim = hidden_dim
        else:
            raise ValueError('Wrong argument type for hidden_dim: {}'.format(hidden_dim))

        if isinstance(activation, str):
            self.activation = [activation] * (self.n_layers - 1)
        elif isinstance(activation, list):
            self.hidden_dim = activation
        else:
            raise ValueError('Wrong argument type for activation: {}'.format(activation))

        self._act_f = []
        for act in self.activation:
            if act == 'lrelu':
                self._act_f.append(lambda x: F.leaky_relu(x, negative_slope=slope))
            elif act == 'relu':
                self._act_f.append(lambda x: torch.nn.ReLU()(x))
            elif act == 'xtanh':
                self._act_f.append(lambda x: self.xtanh(x, alpha=slope))
            elif act == 'sigmoid':
                self._act_f.append(F.sigmoid)
            elif act == 'none':
                self._act_f.append(lambda x: x)
            else:
                ValueError('Incorrect activation: {}'.format(act))

        if self.n_layers == 1:
            _fc_list = [nn.Linear(self.input_dim, self.output_dim)]
        else:
            _fc_list = [nn.Linear(self.input_dim, self.hidden_dim[0])]
            for i in range(1, self.n_layers - 1):
                _fc_list.append(nn.Linear(self.hidden_dim[i - 1], self.hidden_dim[i]))
            _fc_list.append(nn.Linear(self.hidden_dim[self.n_layers - 2], self.output_dim))
        self.fc = nn.ModuleList(_fc_list)
        self.to(self.device)

    @staticmethod
    def xtanh(x, alpha=.1):
        """tanh function plus an additional linear term"""
        return x.tanh() + alpha * x

    def forward(self, x, edge_index):
        h = x
        for c in range(self.n_layers):
            if c == self.n_layers - 1:
                if self.end == 'linear': 
                    h = self.fc[c](h)
                else:
                    h = self.propagate(h, edge_index)
            else:
                h = self.fc[c](h)
                h = F.dropout(h, p=0.5, training=self.training)
                h = self.propagate(h, edge_index)
                if self.normalize: h = F.normalize(h, p=2, dim=1)
                h = self._act_f[c](h)
        return h

model = GCNter(data.num_features, 32, 32, n_layers=2, end='propagate',
                                normalize=True)

In [ ]:
from torch_geometric.utils import (
    add_self_loops,
    negative_sampling,
    remove_self_loops, to_undirected,
     to_dense_adj
)

In [ ]:
from sklearn.decomposition import PCA

from umap import UMAP
def visualize_umap(out, color, size=30, epoch=None, loss = None):
    umap_2d = UMAP(n_components=2, init='random', random_state=0)
    z = umap_2d.fit_transform(out.detach().cpu().numpy())

    plt.figure(figsize=(7,7))
    plt.xticks([])
    plt.yticks([])
    plt.scatter(z[:, 0], z[:, 1], s=size, c=color, cmap="Set2")
    if epoch is not None and loss is not None:
        plt.xlabel(f'Epoch: {epoch}, Loss: {loss:.4f}', fontsize=16)
    plt.show()
    
    
def deg(index, num_nodes = None,           
        dtype = None):
    r"""Computes the (unweighted) degree of a given one-dimensional index tensor.
    Args:
    index (LongTensor): Index tensor.
    num_nodes (int, optional): The number of nodes, *i.e.*
    :obj:`max_val + 1` of :attr:`index`. (default: :obj:`None`)
    dtype (:obj:`torch.dtype`, optional): The desired data type of the
    returned tensor.\n\n    :rtype: :class:`Tensor`\n    """
    if index.shape[0] != 1: # modify input 
        index = index[0] 
    N = num_nodes
    out = torch.zeros((N, ), dtype=dtype, device=index.device)
    one = torch.ones((index.size(0), ), dtype=out.dtype, device=out.device)
    return out.scatter_add_(0, index, one)


In [ ]:
model = GCNter(data.num_features, 32, 32, n_layers=2, end='end',
                                normalize=True)
### we might not want to normalize.
### we might not want to use the dot product as a similarity matrix/
optimizer = torch.optim.Adam(model.parameters(), lr=0.001,
                                 weight_decay=1e-4)
criterion = torch.nn.CrossEntropyLoss()  
from models.baseline_models import LogReg
from sklearn.metrics import accuracy_score
from train_utils import *
from scipy import optimize
from torch_geometric.utils import add_remaining_self_loops, to_scipy_sparse_matrix, from_scipy_sparse_matrix
from torch_scatter import scatter_add 


training_rate = 0.85
MAX_EPOCH_EVAL = 100
val_ratio = (1.0 - training_rate) / 3
test_ratio = (1.0 - training_rate) / 3 * 2
transform = RandomLinkSplit(num_val=val_ratio, num_test=test_ratio,
                                is_undirected=True, split_labels=True)
transform_nodes = RandomNodeSplit(split = 'test_rest',
                                      num_train_per_class = 20,
                                      num_val = 500)
train_data, val_data, test_data = transform(data)
rand_data = transform_nodes(data)
# MIN_DIST = 1e-1  Worked really well
MIN_DIST = 1e-1
EPS = MIN_DIST

edge_index0 = torch.vstack([torch.from_numpy(np.array(new_rows)),
                          torch.from_numpy(np.array(new_cols))]).long() #### work with the diffusion data

P = to_scipy_sparse_matrix(edge_index0, 
                           edge_attr=torch.from_numpy(np.array(weights)), 
                           num_nodes=data.num_nodes)
A = P + P.T - P.multiply(P.T)
d = from_scipy_sparse_matrix(A)#
edge_weights  = d[1]  #0.5 * (1 + d[1])
rows, cols = d[0]
edge_index = torch.vstack([torch.from_numpy(np.array(rows)),
                          torch.from_numpy(np.array(cols))]).long()
#edge_weights = torch.from_numpy(np.array(edge_weights))
print(edge_weights.shape,len(weights), edge_index.shape)

plt.figure()
plt.hist(edge_weights.numpy())
plt.show()


x = np.linspace(0, 3, 300)

def f(x, min_dist):
    y = []
    for i in range(len(x)):
        if(x[i] <= min_dist):
            y.append(1)
        else:
            y.append(np.exp(- x[i] + min_dist))
    return y

dist_low_dim = lambda x, a, b: 1 / (1 + a*x**(2*b))


p , _ = optimize.curve_fit(dist_low_dim, x, f(x, MIN_DIST))
a = p[0]
b = p[1]
print("Hyperparameters a = " + str(a) + " and b = " + str(b))



for epoch in range(140):
    model.train()
    optimizer.zero_grad()

    out = model(data.x, data.edge_index)  ### The GNN is trained on the original data
    train_index = np.arange(data.edge_index.shape[1]) #dattorch.randperm(data.edge_index.shape[1])#[:1000]
    (row_pos, col_pos), edge_weights_pos = remove_self_loops(edge_index, edge_weights) #### try to reconstruct the network based on the diffusion process
    indices = torch.randperm(len(row_pos))[:data.edge_index.shape[1]] ### take a subset, there are too many
    (row_pos, col_pos), edge_weights_pos = (row_pos[indices], col_pos[indices]), edge_weights_pos[indices]
    diff_norm = torch.sum(torch.square(out[row_pos] - out[col_pos]), 1) + EPS
    q =  torch.pow(1.  + a * torch.exp(b * torch.log(diff_norm)), -1)
    q = torch.clamp(q, EPS, 1.-EPS)  ### ensure above 0
    #### Maybe the as and the bs should be node dependent
    edge_weights_pos = (1-EPS) * torch.ones((len(row_pos), 1))
    loss =  - torch.mean(edge_weights_pos *  torch.log(q)  )  - torch.mean((1.-edge_weights_pos) * (  torch.log(1.-q)  )) 
    #print("loss pos", loss)
    neg_edge_index = negative_sampling(data2.edge_index, data2.num_nodes, num_neg_samples=1000)
    row_neg, col_neg = neg_edge_index[0], neg_edge_index[1]
    diff_norm_neg = torch.sum(torch.square(out[row_neg] - out[col_neg]), 1)+ EPS
    q_neg = torch.pow(1.  + a * torch.exp(b * torch.log(diff_norm_neg)), -1)
    #### This might still not be good and too high dimensional!
    q_neg = torch.clamp(q_neg, EPS, 1.-EPS)  ### ensure above 0
#     #print("q_neg", torch.max(q_neg),  torch.min(q_neg))
    edge_weights_neg = EPS * torch.ones((neg_edge_index.shape[1], 1))
    loss +=  torch.mean(EPS * ( - torch.log(q_neg)  ))  + torch.mean((1.-EPS) * ( - torch.log(1.-q_neg)  )) 
    loss.backward()
    
    optimizer.step()
    #print("weight", model.fc[0].weight.grad)

    print('Epoch={:03d}, loss={:.4f}'.format(epoch, loss.item()))
    if epoch % 10 == 0 :
                print("=== Evaluation ===")
                embeds = out
                plt.figure()
                visualize_umap(out, data.y.numpy(), size=30, epoch=None, loss = None)
                plt.show()
                _, res, best_epoch = edge_prediction(embeds.detach(), embeds.shape[1],
                                         train_data, test_data, val_data,
                                         lr=0.01, wd=1e-4,
                                         patience = 30,
                                         max_epochs=MAX_EPOCH_EVAL)
                val_ap, val_roc, test_ap, test_roc, train_ap, train_roc = res[best_epoch][1], res[best_epoch][2], res[best_epoch][3], res[best_epoch][4], res[best_epoch][5], res[best_epoch][6]
                
                _, nodes_res, best_epoch = node_prediction(embeds.detach(),
                                               dataset.num_classes, data.y,
                                               rand_data.train_mask, rand_data.test_mask,
                                               rand_data.val_mask,
                                               lr=0.01, wd=1e-4,
                                               patience = 20,
                                               max_epochs=MAX_EPOCH_EVAL)

                acc_train, val_train, acc = nodes_res[best_epoch][2], nodes_res[best_epoch][3], nodes_res[best_epoch][4]

                _, nodes_res_default, best_epoch = node_prediction(embeds.detach(),
                                               dataset.num_classes, data.y,
                                               data.train_mask, data.test_mask,
                                               data.val_mask,
                                               lr=0.05, wd=0,
                                               patience = 200,
                                               max_epochs=MAX_EPOCH_EVAL)
                acc_train_default, acc_val_default, acc_default = nodes_res_default[best_epoch][2], nodes_res_default[best_epoch][3], nodes_res_default[best_epoch][4]
                print(['ICA', train_roc, train_ap,
                   test_roc, test_ap, acc_train, val_train, acc,
                   acc_train_default, acc_val_default, acc_default, epoch,])
None;

None;

In [ ]:
#### Compute the average radius
for u in range(20):
    index = np.array([e for e in range(len(diff_norm)) if row_pos[e]==u])
    x = diff_norm.detach().numpy()[index]
    y = diff_norm_neg.detach().numpy()[index]
    w = edge_weights.numpy()[index]
    q_ind = q.detach().numpy()[index]
    print(np.mean(np.sqrt(x)), np.std(np.sqrt(x)), np.mean(np.sqrt(x))/np.std(np.sqrt(x)))
    print(np.sum(np.sqrt(x) * q_ind )/np.sum(q_ind ))
    plt.figure()
    plt.hist([x, y], label=['neighbours', 'neg samples'])
    plt.title('nb neighbours = '+ str(len(x)) + ' and average radius: '+str(np.sum(np.sqrt(x) * w)/np.sum(w)) + ' and average radius q: '+str(np.sum(np.sqrt(x) * q_ind)/np.sum(q_ind)))
    plt.show()